<a href="https://colab.research.google.com/github/KosteRico/graph-ontology-2022/blob/master/rdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install rdflib

     |████████████████████████████████| 482 kB 4.5 MB/s 
     |████████████████████████████████| 41 kB 544 kB/s 


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/smartphones.csv")
df = df[['Страна-производитель','Модель', 'Год релиза', 'Поддержка сетей 4G (LTE)',
        'Поддержка сетей 5G', 'Диагональ экрана (дюйм)', 'Разрешение экрана', 'Плотность пикселей',
        'Технология изготовления экрана', 'Частота обновления экрана', 'Емкость аккумулятора',
        'Степень защиты IP', 'Модель процессора', 'Объем оперативной памяти', 'Объем встроенной памяти',
        'Стереодинамики', 'Поддержка быстрой зарядки', 'Поддержка беспроводной зарядки', 'Вес', 'price', 'url']]

# drop same models with different colors
df.drop_duplicates(subset=['Модель', 'Объем оперативной памяти', 'Объем встроенной памяти'], inplace=True)
df['Модель'] = df['Модель'].replace(r' "Как новый"', '', regex=True)
df.drop_duplicates(subset=['Модель'], inplace=True)
df['Бренд'] = df['Модель'].str.split(' ',1).str[0]

df['price'] = df['price'].replace(r'\s+|₽','', regex=True).fillna(0).astype(int)
df['Диагональ экрана (дюйм)'] = df['Диагональ экрана (дюйм)'].replace(r'\s+|"','', regex=True).astype(float)
df['Тип разрешения экрана'] = df['Разрешение экрана'].replace(r'\d+x|х','', regex=True).astype(int)
df['Вес'] = df['Вес'].replace(r'\s+|г','', regex=True).astype(float)
df['Плотность пикселей'] = df['Плотность пикселей'].dropna().replace(r'\s+|ppi','', regex=True).astype(float)
df['Частота обновления экрана'] = df['Частота обновления экрана'].fillna(60).replace(r'\s+|Гц','', regex=True).astype(int)
df['Объем оперативной памяти'] = df[df['Модель']=='Apple iPhone 13 Pro']['Объем оперативной памяти'].fillna(6)
df['Объем оперативной памяти'] = df['Объем оперативной памяти'].fillna(4).replace(r'\s+|МБ|Гб','', regex=True).astype(int)
df['Объем встроенной памяти'] = df['Объем встроенной памяти'].replace(r'\s+|ГБ','', regex=True).astype(int)
df['Аккумулятор'] = df['Емкость аккумулятора'].replace(r'\s+|мА\*ч','', regex=True).fillna(3000).astype(int)
df['Модель процессора'] = df['Модель процессора'].replace(r' MT6(7|8)\w+($|/s)|MediaTek |Apple |HiSilicon ', '', regex=True).replace(r'888\+', '888 Plus', regex=True)
print(df['Модель процессора'])

def dimensClass(x):
  if x in range(480, 720):
    x = 'MD+'
  elif x in range(720, 1080):
    x = 'HD+'
  elif x in range(1080, 1440):
    x = 'Full HD+'
  elif x >= 1440:
    x = 'WQHD'
  return x

def isThereFeature(x):
  if x not in ['нет']:
    x = True
  else:
    x = False
  return x

def weightClass(x):
  if x <= 165.0:
    x = 'Light'
  elif x <= 195.0:
    x = 'Mid'
  else:
    x = 'Heavy'
  return x

def pixelDensityClass(x):
  if x < 250:
    x = 'Bad'
  elif x >= 250 and x < 330:
    x = 'Normal'
  elif x >= 330 and x < 380:
    x = 'Good'
  elif x >= 380:
    x = 'Great'
  return x

df = df[df['Модель процессора'].notna()]
df = df[df['Объем оперативной памяти'].notna()]

df['Страна-производитель'] = df['Страна-производитель'].fillna('Китай')
df['Технология изготовления экрана'] = df['Технология изготовления экрана'].fillna('IPS')

df['Тип разрешения экрана'] = df['Тип разрешения экрана'].apply(dimensClass)
df['Класс плотности пикселей'] = df['Плотность пикселей'].apply(pixelDensityClass)
df['Класс веса'] = df['Вес'].apply(weightClass)

for col in ['Поддержка сетей 4G (LTE)', 'Поддержка сетей 5G', 'Степень защиты IP',
            'Стереодинамики', 'Поддержка быстрой зарядки', 'Поддержка беспроводной зарядки']:
  df[col] = df[col].fillna('нет').apply(isThereFeature)

df = df.rename(columns={'Модель': 'name'})

print(df)
print(df.isna().sum())
print(df.dtypes)

0                     MT6580
1         Spreadtrum SC7731E
4                    MT6580M
6                    MT6580M
7                    MT6580M
                ...         
1102          Snapdragon 855
1103          Snapdragon 865
1105    Exynos 7 Series 7884
1108      Spreadtrum SC9832E
1113               Helio P70
Name: Модель процессора, Length: 382, dtype: object
     Страна-производитель                name  Год релиза  \
0                   Китай         INOI 1 Lite        2018   
1                   Китай           DEXP A440        2021   
4                   Китай           DEXP G450        2020   
6                   Китай       DEXP A350 MIX        2020   
7                   Китай           DEXP A250        2020   
...                   ...                 ...         ...   
1102              Таиланд       Sony Xperia 5        2019   
1103              Таиланд    Sony Xperia 5 II        2020   
1105              Вьетнам  Samsung Galaxy A10        2019   
1108               

In [ ]:
import pandas as pd
df_proc = pd.read_csv("/content/drive/MyDrive/processors.csv")
df_proc = df_proc[['name', 'rating_score', 'rating_mark']].rename(columns={'name': 'Модель процессора', 'rating_score': 'proc_score', 'rating_mark': 'proc_mark'})
df_auto = pd.read_csv("/content/drive/MyDrive/battery.csv")
df_auto = df_auto[['name', 'rating']].rename(columns={'rating': 'autonomy_score'})
df_camera = pd.read_csv("/content/drive/MyDrive/camera.csv")
df_camera = df_camera[['name', 'camera_score', 'camera_rate']]

print(df_proc)
df_auto['name'] = df_auto['name'].replace(r'Oppo Rea', 'Rea', regex=True).replace(r'Xiaomi Po', 'Po', regex=True).replace(r' (Snapdragon)', '', regex=True).replace(r'\+',' Plus',regex=True)
df['name'] = df['name'].replace('OPPO', 'Oppo', regex=True).replace(r'HUAWEI', 'Huawei', regex=True).replace(r'Huawei Ho', 'Ho', regex=True).replace(r'\+',' Plus',regex=True)
print(df_auto)
df_camera['name'] = df_camera['name']

res = df.merge(df_proc, on=["Модель процессора"], how="left")
res = res.merge(df_auto, on=["name"], how="left")
res = res.merge(df_camera, on=["name"], how="left")
print(res)
print(res.isna().sum())
res['proc_score'] = res['proc_score'].fillna(10)
res['proc_mark'] = res['proc_mark'].fillna('D')
res['autonomy_score'] = res['autonomy_score'].fillna(10)
res['camera_score'] = res['camera_score'].fillna(10)
res['camera_rate'] = res['camera_rate'].fillna(1000)
#print(res[res['autonomy_score'] > 10])

      Модель процессора  proc_score proc_mark
0            A15 Bionic          97        A+
1        Dimensity 9000          95        A+
2    Snapdragon 8 Gen 1          94        A+
3            A14 Bionic          93        A+
4           Exynos 2200          91        A+
..                  ...         ...       ...
120           Helio P22          22         D
121      Snapdragon 450          22         D
122           Helio P23          21         D
123           Kirin 658          21         D
124         Exynos 7880          21         D

[125 rows x 3 columns]
                    name  autonomy_score
0              Realme 6i             113
1     Samsung Galaxy M51             105
2               Realme 5             103
3     Samsung Galaxy A42             102
4              Realme 5s             102
..                   ...             ...
120             Realme 3              74
121    Sony Xperia 5 III              73
122    OnePlus Nord 2 5G              73
123  Asus Zenf

In [ ]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.serializer import Serializer

# Create a Graph
g = Graph()

g.parse("/content/drive/MyDrive/smart-3-1.owl")
# Create an RDF URI node to use as the subject for multiple triples
n = Namespace("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1")

res['name'] = res['name'].replace(r' ', '_', regex=True)
res['Страна-производитель'] = res['Страна-производитель'].replace(r', республика| \(китай\)', '', regex=True)
res['Название процессора'] = res['Модель процессора']
res['Модель процессора'] = res['Модель процессора'].replace(r' ', '_', regex=True)
res['Технология изготовления экрана'] = res['Технология изготовления экрана'].replace(r' ', '_', regex=True)

#Classes
Smartphone_class = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#Smartphone")
Country_class = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#Country")
Company_class = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#Company")
Chip_class = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#Chip")
Display_class = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#Display")

#Object properties
hasCompany = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#hasCompany")
isCompanyOf = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#isCompanyOf")
hasCountry = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#hasCountry")
isCountryOf = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#isCountryOf")
hasChip = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#hasChip")
isChipOf = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#isChipOf")
hasDisplay = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#hasDisplay")
isDisplayOf = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#isDisplayOf")

#Data properties
has4G = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#4G")
has5G = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#5G")
fastCharge = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#fastCharge")
wirelessCharge = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#wirelessCharge")
protection = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#protection")
stereoSound = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#stereoSound")
autonomyScore = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#autonomyScore")
cameraScore = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#cameraScore")
modelName = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#modelName")
price = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#price")
url = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#url")
weight = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#weight")
weightClass = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#weightClass")
year = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#year")
RAM = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#RAM")
ROM = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#ROM")
battery = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#battery")

chipName = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#chipName")
chipClass = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#chipClass")
chipScore = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#chipScore")

diagonal = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#diagonal")
displayType = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#displayType")
density = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#density")
densityRank = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#densityRank")
frequency = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#frequency")
resolution = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#resolution")
resolutionType = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#resolutionType")

companyName = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#companyName")
countryName = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#countryName")

# Add triples using store's add() method.
for index, row in res.iterrows():
  smartphone = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone#"+row['name'])
  country = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone#"+row['Страна-производитель'])
  company = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone#"+row['Бренд'])
  chip = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone#"+row['Модель процессора'])
  display = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone#"+row['Технология изготовления экрана']+'_'
                    +row['Разрешение экрана'])+'_'+str(row['Диагональ экрана (дюйм)'])+'_'+str(row['Частота обновления экрана'])

  #Classes
  g.add((smartphone, RDF.type, Smartphone_class))
  g.add((country, RDF.type, Country_class))
  g.add((company, RDF.type, Company_class))
  g.add((chip, RDF.type, Chip_class))
  g.add((display, RDF.type, Display_class))
  
  #Object properties
  g.add((smartphone, hasCompany, company))
  g.add((company, isCompanyOf, smartphone))
  g.add((company, hasCountry, country))
  g.add((country, isCountryOf, company))
  g.add((smartphone, hasChip, chip))
  g.add((chip, isChipOf, smartphone))
  g.add((smartphone, hasDisplay, display))
  g.add((display, isDisplayOf, smartphone))

  #Data properties
  g.add((smartphone, has4G, Literal(row['Поддержка сетей 4G (LTE)'], datatype=XSD.boolean)))
  g.add((smartphone, has5G, Literal(row['Поддержка сетей 5G'], datatype=XSD.boolean)))
  g.add((smartphone, fastCharge, Literal(row['Поддержка быстрой зарядки'], datatype=XSD.boolean)))
  g.add((smartphone, wirelessCharge, Literal(row['Поддержка беспроводной зарядки'], datatype=XSD.boolean)))
  g.add((smartphone, protection, Literal(row['Степень защиты IP'], datatype=XSD.boolean)))
  g.add((smartphone, stereoSound, Literal(row['Стереодинамики'], datatype=XSD.boolean)))
  g.add((smartphone, autonomyScore, Literal(row['autonomy_score'], datatype=XSD.integer)))
  g.add((smartphone, cameraScore, Literal(row['camera_score'], datatype=XSD.integer)))
  g.add((smartphone, modelName, Literal(row['name'])))
  g.add((smartphone, price, Literal(row['price'], datatype=XSD.integer)))
  g.add((smartphone, url, Literal(row['url'])))
  g.add((smartphone, weight, Literal(row['Вес'], datatype=XSD.float)))
  g.add((smartphone, weightClass, Literal(row['Класс веса'])))
  g.add((smartphone, year, Literal(row['Год релиза'], datatype=XSD.integer)))
  g.add((smartphone, RAM, Literal(row['Объем оперативной памяти'], datatype=XSD.integer)))
  g.add((smartphone, ROM, Literal(row['Объем встроенной памяти'], datatype=XSD.integer)))
  g.add((smartphone, battery, Literal(row['Аккумулятор'], datatype=XSD.integer)))
  g.add((chip, chipName, Literal(row['Название процессора'])))
  g.add((chip, chipClass, Literal(row['proc_mark'])))
  g.add((chip, chipScore, Literal(row['proc_score'], datatype=XSD.integer)))
  g.add((company, companyName, Literal(row['Бренд'])))
  g.add((country, countryName, Literal(row['Страна-производитель'])))
  g.add((display, diagonal, Literal(row['Диагональ экрана (дюйм)'], datatype=XSD.float)))
  g.add((display, displayType, Literal(row['Технология изготовления экрана'])))
  g.add((display, frequency, Literal(row['Частота обновления экрана'], datatype=XSD.integer)))
  g.add((display, resolution, Literal(row['Разрешение экрана'])))
  g.add((display, resolutionType, Literal(row['Тип разрешения экрана'])))
  g.add((display, density, Literal(row['Плотность пикселей'], datatype=XSD.integer)))
  g.add((display, densityRank, Literal(row['Класс плотности пикселей'])))

# Add another person
#ed = URIRef("http://example.org/edward")
#name = URIRef("http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1/" + 'model')

res.to_csv("/content/drive/MyDrive/sm.csv")
g.serialize(destination="/content/drive/MyDrive/smart-3-2.rdf", format='application/rdf+xml')

In [ ]:
!pip install owlrl

     |████████████████████████████████| 54 kB 2.4 MB/s 


In [ ]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph

g = Graph()

# Parse in an RDF file hosted on the Internet
g.parse("/content/drive/MyDrive/smart.owl", format='n3')

<Graph identifier=N110516b0849c4c9ba02546b50e9546cb (<class 'rdflib.graph.Graph'>)>

In [ ]:
prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX smt: <http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#>
"""

queries = [
"""
SELECT ?subject
    WHERE { ?subject rdf:type smt:ForBlogging ;
                     smt:price ?price ;
                     smt:5G ?5g ;
                 FILTER ( ?price > 50000 && ?5g = true )

     }
""",
"""
SELECT ?subject
    WHERE { ?subject rdf:type smt:ForGaming ;
                     rdf:type smt:GoodAutonomy
     }
""",
"""
SELECT ?subject
    WHERE { ?subject rdf:type smt:ForGaming ;
                     smt:price ?price .
                 FILTER ( ?price < 30000 )
     }
""",
"""
SELECT ?subject
    WHERE { ?subject rdf:type smt:ForFun ;
                     smt:price ?price ;
                     smt:hasCompany ?company .
            ?company smt:companyName ?companyName
                 FILTER ( ?companyName = "OPPO" && ?price < 30000 )
     }
""",
"""
SELECT DISTINCT ?subject
    WHERE { ?subject rdf:type smt:ForStudy ;
                     smt:hasCompany ?company .
            ?company smt:hasCountry ?country .
                 FILTER ( regex(str(?country), ".*#Китай"))
     }
""",
"""
SELECT ?subject
    WHERE { ?subject  rdf:type smt:Compact ;
                      rdf:type smt:GreatPerfomance .
     }
""",
"""
SELECT ?subject ?price
    WHERE { ?subject  rdf:type smt:GreatAutonomy ;
                      smt:price ?price ;
                      smt:hasDisplay ?display .
            ?display smt:resolutionType ?resType .
            FILTER ( ?resType = "WQHD" )
      } ORDER BY ASC (?price)
""",
"""
SELECT ?subject
    WHERE { ?subject  rdf:type smt:ForFun ;
                      smt:year ?year .
            FILTER ( ?year > 2020 )
      }
""",
"""
SELECT ?subject
    WHERE { ?subject   rdf:type smt:ForElderly ;
                       rdf:type smt:ForStudy .
    }
""",
"""
SELECT (COUNT(*) as ?cnt) WHERE { ?s ?p ?o }
"""
]

queries = [prefixes + q for q in queries]
queries

['\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX smt: <http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#>\n\nSELECT ?subject\n    WHERE { ?subject rdf:type smt:ForBlogging ;\n                     smt:price ?price ;\n                     smt:5G ?5g ;\n                 FILTER ( ?price > 50000 && ?5g = true )\n\n     }\n',
 '\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX smt: <http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#>\n\nSELECT ?subject\n    WHERE { ?subject rdf:type smt:ForGaming ;\n                     rdf:type smt:GoodAutonomy\n     }\n',
 '\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX owl:

In [ ]:

for q in queries:
  qres = g.query(q)

  print(f'SPARQL query "{q}"\n')

  print('RESULT:')
  for row in qres:
    print(str(row))

  print('\n' + '-'*50 + '\n')


SPARQL query "
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX smt: <http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#>

SELECT ?subject
    WHERE { ?subject rdf:type smt:ForBlogging ;
                     smt:price ?price ;
                     smt:5G ?5g ;
                 FILTER ( ?price > 50000 && ?5g = true )

     }
"

RESULT:

--------------------------------------------------

SPARQL query "
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX smt: <http://www.semanticweb.org/eugene/ontologies/2022/1/smartphone-1#>

SELECT ?subject
    WHERE { ?subject rdf:type smt:ForGaming ;
                     rdf:type smt:GoodAutonomy
     }
"

RESULT:

-----